In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import pandas as pd
import logging
import os, sys
from tqdm.notebook import tqdm
# sys.path.append(r"C:\Users\chataint\Documents\projet\humanlisbet\lisbet_code")
# from utils import load_embedding


In [2]:
data_r = r"C:\Users\chataint\Documents\projet\humanlisbet\results\bet_embedders\bet_embedders\13879972"
seed = 42

In [8]:
def load_embedding(datapath, window_size=200, stride=30):
    emb_train = np.load(datapath, allow_pickle=True)

    # Initialize an empty list to collect rows for the DataFrame
    rows = []

    # Iterate over the array to flatten the structure
    for video, coords in emb_train:
        for idx, coord in enumerate(coords):
            rows.append([video, idx] + coord.tolist())


    for video_id, video_data in emb_train:
        # diag = video_data.iloc[0].diagnosis
        # vd = video_data.drop(['video', 'diagnosis'], axis=1).to_numpy()
        num_frames= len(video_data)
        # Create windows
        for start in range(0, num_frames - window_size + 1, stride):
            window = video_data[start : start + window_size]
            rows.append([video] + window.tolist())

    # Create a DataFrame
    df = pd.DataFrame(rows, columns=['video'] + [f'em_{i}' for i in range(len(coord))])
    df.video = df.video.astype(np.int64)


    logging.info(f"Data loaded from {datapath}")

    return df

In [9]:

# datapath = os.path.join(data_r, "embedding_train.numpy")
dataval = os.path.join(data_r, "embedding_test.numpy")

# Load your temporal encoded data (shape: n_samples, 128 dimensions)
# train_data =  load_embedding(datapath)# Replace with your data
# td = train_data.drop(columns='video').to_numpy()

eval_data = load_embedding(dataval)
te = eval_data.drop(columns='video').to_numpy()

# td = td[:100]
# te = te[:100] 


ValueError: 129 columns passed, passed data had 201 columns

In [7]:
te.shape

(996069, 128)

In [8]:
# save the data
np.save(os.path.join(data_r, "train_test.npy"), td)
np.save(os.path.join(data_r, "eval_testnpy"), te)

In [8]:
np.load(os.path.join(data_r, "train_test.npy"))
df = pd.DataFrame(td)
df.rename(columns={i: f"em_{i}" for i in range(128)}, inplace=True)
df['video'].map()

AttributeError: module 'umap' has no attribute 'UMAP'

In [4]:

# Step 1: Dimensionality Reduction with UMAP
data_umap = umap_reducer.fit_transform(td)

c:\Users\chataint\AppData\Local\anaconda3\envs\diffversify\lib\site-packages\sklearn\manifold\_spectral_embedding.py:392: UserWarning: Exited at iteration 526 with accuracies 
[1.15224576e-15 2.34491319e-05 2.62052876e-05 2.40631266e-05]
not reaching the requested tolerance 2.4721026420593262e-05.
Use iteration 427 instead with accuracy 
1.509689262591631e-05.

  _, diffusion_map = lobpcg(
c:\Users\chataint\AppData\Local\anaconda3\envs\diffversify\lib\site-packages\sklearn\manifold\_spectral_embedding.py:392: UserWarning: Exited postprocessing with accuracies 
[1.47487181e-15 2.08817564e-05 1.33297035e-05 2.61761107e-05]
not reaching the requested tolerance 2.4721026420593262e-05.
  _, diffusion_map = lobpcg(


KeyboardInterrupt: 

In [ ]:
# Step 2: K-Means Clustering and Finding the Optimal Number of Clusters
def kmeans_clustering(data, max_clusters=10, min_clusters=2, step=1):
    inertia = []
    silhouette = []

    for k in tqdm(range(min_clusters, max_clusters + 1, step)):
        kmeans = KMeans(n_clusters=k, random_state=seed)
        labels = kmeans.fit_predict(data)
        inertia.append(kmeans.inertia_)
        silhouette.append(silhouette_score(data, labels))
    return inertia, silhouette

# Run clustering for up to 10 clusters
min_clusters = 5
max_clusters = 100
step=5
inertia, silhouette = kmeans_clustering(data_umap, max_clusters, min_clusters, step)

In [ ]:
# Step 3: Plot Inertia and Silhouette Scores
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].plot(range(2, max_clusters + 1), inertia, marker='o')
ax[0].set_title("Elbow Method (Inertia)")
ax[0].set_xlabel("Number of Clusters")
ax[0].set_ylabel("Inertia")

ax[1].plot(range(2, max_clusters + 1), silhouette, marker='o')
ax[1].set_title("Silhouette Score")
ax[1].set_xlabel("Number of Clusters")
ax[1].set_ylabel("Silhouette Score")

plt.show()

In [ ]:
# Step 4: Visualization of Clusters
optimal_clusters = silhouette.index(max(silhouette)) + 2
kmeans = KMeans(n_clusters=optimal_clusters, random_state=seed)
labels = kmeans.fit_predict(data_umap)

plt.scatter(data_umap[:, 0], data_umap[:, 1], c=labels, cmap='viridis', s=10)
plt.title(f"K-Means Clustering on UMAP with {optimal_clusters} Clusters")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.colorbar(label="Cluster Label")
plt.show()
